### 모델 라이브러리

In [57]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

In [76]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000)
        , 'num_leaves': trial.suggest_int('num_leaves', 300, 700)
        , 'max_depth': trial.suggest_int('max_depth', 10, 30)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1)
        , 'min_child_samples': trial.suggest_int('min_child_samples', 3, 30)
  
        , 'boosting' : 'dart' # dart 사용
        , 'random_state': 0
        , 'verbose' : -1
    }
    
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"].astype(int),
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-02-22 16:58:35,039] A new study created in memory with name: no-name-f0e8ef9a-e7ab-464f-b92f-ae8e9eeae3ea
[I 2024-02-22 16:59:03,932] Trial 0 finished with value: 0.8371572467823168 and parameters: {'n_estimators': 684, 'num_leaves': 586, 'max_depth': 22, 'learning_rate': 0.05494343511669279, 'min_child_samples': 14}. Best is trial 0 with value: 0.8371572467823168.
[I 2024-02-22 16:59:33,959] Trial 1 finished with value: 0.8339943342776204 and parameters: {'n_estimators': 752, 'num_leaves': 475, 'max_depth': 28, 'learning_rate': 0.09640261328960191, 'min_child_samples': 13}. Best is trial 0 with value: 0.8371572467823168.
[I 2024-02-22 17:00:08,291] Trial 2 finished with value: 0.831908831908832 and parameters: {'n_estimators': 854, 'num_leaves': 512, 'max_depth': 21, 'learning_rate': 0.09263406719097345, 'min_child_samples': 4}. Best is trial 0 with value: 0.8371572467823168.
[I 2024-02-22 17:00:17,288] Trial 3 finished with value: 0.8312849162011172 and parameters: {'n_estima

Best trial: score 0.845117845117845, 
params {'n_estimators': 301, 'num_leaves': 383, 'max_depth': 15, 'learning_rate': 0.050832163879494394, 'min_child_samples': 4}


### 모델 성능확인

In [58]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [78]:
# LightGBM_dart
model = LGBMClassifier(
    n_estimators=301
    , num_leaves=383
    , max_depth=15
    , learning_rate=0.050832163879494394
    , min_child_samples=4
    , boosting='dart'  # dart 사용
    , random_state=0
    , verbose=-1
)

### 모델 학습

In [79]:
model.fit(x_train, y_train)

LGBMClassifier(boosting='dart', learning_rate=0.050832163879494394,
               max_depth=15, min_child_samples=4, n_estimators=301,
               num_leaves=383, random_state=0, verbose=-1)

In [80]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,753,194
False,82,10831


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.976728,0.901796,0.795143,0.845118,0.976057


In [81]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

684

### 제출파일 생성

In [82]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_lgbm_dart_optuna.csv", index=False)

.